In [1]:
import logging, logging.handlers
import sys
import pickle
import numpy as np
from collections import namedtuple
from itertools import count
import random
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.autograd as autograd
from torch.autograd import Variable

import seaborn as sns; sns.set(color_codes=True)
%matplotlib inline
import matplotlib.ticker as mtick
import matplotlib.pyplot as plt
import torchvision.utils as vutils
import math

# round_ = 1
# lookahead = 1
target_update = 10000
w = 10
h = 10
life_cost = 0.05
# env_name = 'Grid'
# env_name = '%s_LA_%d_W%d_H%d_round%d' %(env_name,lookahead,w,h,round_)
# logger = logging.getLogger()
# file_name = './data/results_%s.log' %(env_name)
# fh = logging.handlers.RotatingFileHandler(file_name)
# fh.setLevel(logging.DEBUG)#no matter what level I set here
# formatter = logging.Formatter('%(asctime)s:%(message)s')
# fh.setFormatter(formatter)
# logger.addHandler(fh)

In [2]:
batchsize = 64
USE_CUDA = torch.cuda.is_available()
dtype = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.FloatTensor
seed = 0
number_action = 4
dimension = 2

import torch
import torch.nn as nn
import torch.nn.functional as F
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv' or 'SNConv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)
        
class DQN(nn.Module):
    def __init__(self, dimension=2, num_actions=number_action):
        super(DQN, self).__init__()
        self.fc1 = nn.Linear(dimension, 50)
#         self.fc2 = nn.Linear(500, 500)
        self.fc3 = nn.Linear(50, num_actions)
    def forward(self, x):
        x = F.relu(self.fc1(x.view(x.size(0), -1)))
#         x = F.relu(self.fc2(x.view(x.size(0), -1)))
        return self.fc3(x)


In [3]:
w_ = w-1
h_ = 0
h__ = h-1
gamma = 0.99
pos_rew = 1
def recurs(imagine_,h,Q):
    if h == 0:
        val = Q(imagine_.unsqueeze(0).unsqueeze(0).type(dtype)).max(1)
        return val[0], val[1][0]
    else:
        action = 0
        for i in range(number_action):
            imagine = torch.clone(imagine_)
            if i == 0:
                imagine[1] = torch.clone(torch.clamp(imagine_[1] - 1,0,h-1))
                cum_return_ = gamma * recurs(imagine,h-1,Q)[0] + pos_rew * torch.all(torch.eq(imagine, torch.tensor([[w_,h_]],dtype=torch.int8))).type(torch.FloatTensor)\
                                                               - torch.all(torch.eq(imagine[1], torch.tensor([h__],dtype=torch.int8))).type(torch.FloatTensor) - life_cost
                
            if i == 1:
                imagine[0] = torch.clone(torch.clamp(imagine_[0] - 1,0,w-1))
                cum_return = gamma * recurs(imagine,h-1,Q)[0] + pos_rew * torch.all(torch.eq(imagine, torch.tensor([[w_,h_]],dtype=torch.int8))).type(torch.FloatTensor)\
                                                              - torch.all(torch.eq(imagine[1], torch.tensor([h__],dtype=torch.int8))).type(torch.FloatTensor) - life_cost
                if cum_return > cum_return_:
                    cum_return_ = torch.clone(cum_return)
                    action = 1
            if i == 2:
                imagine[1] = torch.clone(torch.clamp(imagine_[1] + 1,0,h-1))
                cum_return = gamma * recurs(imagine,h-1,Q)[0] + pos_rew * torch.all(torch.eq(imagine, torch.tensor([[w_,h_]],dtype=torch.int8))).type(torch.FloatTensor)\
                                                              - torch.all(torch.eq(imagine[1], torch.tensor([h__],dtype=torch.int8))).type(torch.FloatTensor) - life_cost
                if cum_return > cum_return_:
                    cum_return_ = torch.clone(cum_return)
                    action = 2
            if i == 3:
                imagine[0] = torch.clone(torch.clamp(imagine_[0] + 1,0,w-1))
                cum_return = gamma * recurs(imagine,h-1,Q)[0] + pos_rew * torch.all(torch.eq(imagine, torch.tensor([[w_,h_]],dtype=torch.int8))).type(torch.FloatTensor)\
                                                              - torch.all(torch.eq(imagine[1], torch.tensor([h__],dtype=torch.int8))).type(torch.FloatTensor) - life_cost
                if cum_return > cum_return_:
                    cum_return_ = torch.clone(cum_return)
                    action = 3   
#         print(cum_return_)            
        return cum_return_, torch.tensor(action)
                         

In [ ]:
#action
#  0
#1   3
#  2
#         w
# x=0,y=0    x=w,y=0    
#                      h
# x=0,y=h    x=w,y=h   

for round_count in range(10):
    round_ = round_count + 1
    lookahead = 2
    env_name = 'Grid'
    env_name = '%s_LA_%d_W%d_H%d_round%d' %(env_name,lookahead,w,h,round_)

    init_w = 0
    init_h = h-1
    tot_epis = 1000
    epsilon = .1


    Q = DQN(dimension,number_action).type(dtype) 
    Target_Q = DQN(dimension,number_action).type(dtype) 
    Q.apply(weights_init)
    optimizer = optim.SGD(Q.parameters(), lr = .0001, momentum=0.9)


    actions = torch.zeros((1),dtype=torch.int8)
    states = torch.zeros((1,2),dtype=torch.int8)
    # done = torch.zeros((1)).type(dtype) 
    # states = torch.tensor([init_w,init_h])
    rewards = torch.zeros((1)).type(dtype) 
    tot_rew = torch.zeros((1)).type(dtype) 
    # next_states = torch.zeros((1,2),dtype=torch.int8)
    e = 0
    tot_time = 2100000
    time = 0
    while time < tot_time:
        t = 0
        goal = 0
        cum_rew = 0
        # Initiate the state
        if time > 0:
            states = torch.cat((states,torch.tensor([[0,0]],dtype=torch.int8)),0)
        state = torch.clone(states[-1])
        while t<100 and goal==0:
            time = time+1
            next_state =  torch.clone(state)
            sample = random.random()
            if sample < epsilon:
                action = torch.tensor(random.randrange(number_action))
            else:
                action = recurs(state,lookahead,Q)[1]
            if time == 1:
                actions[0] = action.unsqueeze(0).type(dtype) 
            else:
                actions = torch.cat((actions,action.unsqueeze(0).type(torch.int8)),0)
            reward = pos_rew*torch.all(torch.eq(state, torch.tensor([[w_,h_]],dtype=torch.int8))).type(torch.FloatTensor)\
                    - torch.all(torch.eq(state[1], torch.tensor([h__],dtype=torch.int8))).type(torch.FloatTensor) - life_cost
    #         print(state)
            if reward != -1 * life_cost :
                goal = 1
#                 if reward == pos_rew - life_cost:
#                     print('bingo:t:',t,state)

            if time == 1:
                rewards[0] = reward.unsqueeze(0)
            else:
                rewards = torch.cat((rewards,reward.unsqueeze(0)),0)
            if action == 0:
                next_state[1] = torch.clone(torch.clamp(state[1] - 1,0,h-1))
            if action == 1:
                next_state[0] = torch.clone(torch.clamp(state[0] - 1,0,w-1))
            if action == 2:
                next_state[1] = torch.clone(torch.clamp(state[1] + 1,0,h-1))
            if action == 3:
                next_state[0] = torch.clone(torch.clamp(state[0] + 1,0,w-1))
    #         print(state.type(torch.int).numpy(),action.type(torch.int).numpy(),next_state.type(torch.int).numpy()) 
            cum_rew = cum_rew + gamma * torch.clone(reward)   
            states = torch.cat((states,torch.clone(next_state.unsqueeze(0))),0)
            state = torch.clone(next_state)


    #         current_value = Q(state.unsqueeze(0).unsqueeze(0).type(dtype))[0][action]    
    #         loss = current_value-reward.type(dtype)-(1-goal)*gamma*Target_Q(next_state.unsqueeze(0).unsqueeze(0).type(dtype)).max(1)[0].detach()
    #         d_error = loss.clamp(-1, 1)[0]
    #         optimizer.zero_grad()
    #         current_value.backward(d_error.data)
    #         optimizer.step()
            if time>5000:
                if time % 4 == 0:        
                    choices = np.random.choice(time, batchsize)
                    current_action = actions[choices].unsqueeze(1)
                    current_rewards = rewards[choices].type(dtype)
                    indecies = np.linspace(0,batchsize-1,batchsize,dtype=np.int32)
                    current_nextstates = states[choices+1].unsqueeze(1).type(dtype)
                    current_value = Q(states[choices].unsqueeze(1).type(dtype))[indecies,current_action.squeeze(1).type(torch.int32).numpy()]
                    loss = current_value-current_rewards-(current_rewards== -1 * life_cost).type(dtype)*gamma*Target_Q(current_nextstates).max(1)[0].detach()
                    d_error = loss.clamp(-1, 1)
                    optimizer.zero_grad()
                    current_value.backward(d_error.data)
                    optimizer.step()
            t = t+1

            if time % target_update == 0:
                Target_Q.load_state_dict(Q.state_dict())


        if len(tot_rew) == 1:
            tot_rew[0] = cum_rew.unsqueeze(0).type(dtype)
        else:
            tot_rew = torch.cat((tot_rew,cum_rew.unsqueeze(0).type(dtype)),0) 
        if e%100==0:    
            print('e:',e,'t:',time,'rew:',cum_rew.type(dtype).numpy())
        e = e + 1
    #     s = torch.zeros((1,2),dtype=torch.int32)
    #     s[0,0] = 3
    #     s[0,1] = 3
    #     ss = torch.zeros((1,2),dtype=torch.int32)
    #     ss[0,0] = 0
    #     ss[0,1] = 0
    #     print(Q(s.type(dtype))-Q(ss.type(dtype)))


    fnam = './data/reward%s' %(env_name)
    np.save(fnam,rewards.numpy())                          




    #                 for i2 in range(number_action):







    #  0
    #1   3
    #  2 

e: 0 t: 14 rew: 0.2970001
e: 100 t: 1134 rew: 0.2970001
e: 200 t: 2268 rew: 0.49500006
e: 300 t: 3373 rew: 0.3960001
e: 400 t: 4475 rew: 0.3960001
e: 500 t: 5588 rew: 0.49500006
e: 600 t: 6750 rew: 0.3465001
e: 700 t: 7999 rew: 0.49500006
e: 800 t: 9166 rew: 0.2970001
e: 900 t: 10363 rew: 0.2970001
e: 1000 t: 11649 rew: 0.49500006
e: 1100 t: 12987 rew: 0.44550008
e: 1200 t: 14330 rew: 0.2970001
e: 1300 t: 15683 rew: 0.3960001
e: 1400 t: 17022 rew: 0.24750012
e: 1500 t: 18436 rew: 0.3960001
e: 1600 t: 19878 rew: 0.2970001
e: 1700 t: 21658 rew: -4.9499993
e: 1800 t: 23367 rew: 0.2970001
e: 1900 t: 24973 rew: 0.099000156
e: 2000 t: 26992 rew: 0.19800013
e: 2100 t: 28688 rew: 0.24750012
e: 2200 t: 30631 rew: -4.9499993
e: 2300 t: 38040 rew: -1.7324996
e: 2400 t: 47125 rew: -4.9499993
e: 2500 t: 57073 rew: -4.9499993
e: 2600 t: 66999 rew: -4.9499993
e: 2700 t: 76999 rew: -4.9499993
e: 2800 t: 86961 rew: -4.9499993
e: 2900 t: 96961 rew: -4.9499993
e: 3000 t: 106908 rew: -4.9499993
e: 3100 t:

e: 24000 t: 1063612 rew: -3.5639994
e: 24100 t: 1067689 rew: -1.8809996
e: 24200 t: 1072803 rew: -0.49499977
e: 24300 t: 1078206 rew: -2.9699993
e: 24400 t: 1082197 rew: -1.0889997
e: 24500 t: 1085758 rew: -1.1384997
e: 24600 t: 1089488 rew: 0.099000156
e: 24700 t: 1093234 rew: -0.19799984
e: 24800 t: 1095876 rew: 0.3960001
e: 24900 t: 1099633 rew: 0.2970001
e: 25000 t: 1102752 rew: 0.2970001
e: 25100 t: 1104479 rew: 0.099000156
e: 25200 t: 1106288 rew: 0.2970001
e: 25300 t: 1108280 rew: -0.2969998
e: 25400 t: 1110325 rew: 0.24750012
e: 25500 t: 1112340 rew: 0.14850014
e: 25600 t: 1114246 rew: -0.8909997
e: 25700 t: 1116109 rew: -1.3859997
e: 25800 t: 1117920 rew: 0.099000156
e: 25900 t: 1119896 rew: 0.099000156
e: 26000 t: 1122222 rew: -0.49499977
e: 26100 t: 1124296 rew: 0.24750012
e: 26200 t: 1126912 rew: -0.94049966
e: 26300 t: 1128870 rew: 1.7881393e-07
e: 26400 t: 1130561 rew: 0.3465001
e: 26500 t: 1132139 rew: 0.19800013
e: 26600 t: 1133720 rew: -0.09899986
e: 26700 t: 1135282 r

e: 46700 t: 2082429 rew: -0.6929997
e: 46800 t: 2087440 rew: 0.049500167
e: 46900 t: 2093051 rew: -1.5839996
e: 47000 t: 2098122 rew: -0.6929997
e: 0 t: 14 rew: 0.2970001
e: 100 t: 1359 rew: 0.3960001
e: 200 t: 2689 rew: 0.3960001
e: 300 t: 3977 rew: 0.3960001
e: 400 t: 5500 rew: -4.9499993
e: 500 t: 11221 rew: 0.3960001
e: 600 t: 13824 rew: 0.19800013
e: 700 t: 17037 rew: 0.3960001
e: 800 t: 19165 rew: -3.2174995
e: 900 t: 21019 rew: 0.2970001
e: 1000 t: 22596 rew: 0.2970001
e: 1100 t: 24228 rew: 0.19800013
e: 1200 t: 26136 rew: 0.2970001
e: 1300 t: 28208 rew: 0.3960001
e: 1400 t: 30069 rew: 0.2970001
e: 1500 t: 32772 rew: 0.3960001
e: 1600 t: 35672 rew: 0.2970001
e: 1700 t: 38404 rew: -4.9499993
e: 1800 t: 40793 rew: -4.9499993
e: 1900 t: 44143 rew: -1.6334996
e: 2000 t: 47615 rew: -2.0789995
e: 2100 t: 50290 rew: 0.19800013
e: 2200 t: 53135 rew: 0.3960001
e: 2300 t: 55874 rew: -4.9499993
e: 2400 t: 59116 rew: 0.3960001
e: 2500 t: 66871 rew: -4.9499993
e: 2600 t: 74189 rew: -0.643499

e: 23600 t: 1108207 rew: 0.2970001
e: 23700 t: 1110627 rew: -1.1879997
e: 23800 t: 1112807 rew: 0.3960001
e: 23900 t: 1114503 rew: 0.2970001
e: 24000 t: 1116688 rew: 0.3960001
e: 24100 t: 1118723 rew: -0.19799984
e: 24200 t: 1121201 rew: -4.9499993
e: 24300 t: 1126057 rew: -1.9799995
e: 24400 t: 1129507 rew: 0.2970001
e: 24500 t: 1132416 rew: -1.3859997
e: 24600 t: 1134714 rew: -2.9699993
e: 24700 t: 1138282 rew: 0.3960001
e: 24800 t: 1141553 rew: 0.24750012
e: 24900 t: 1145727 rew: 0.3465001
e: 25000 t: 1149649 rew: 0.3960001
e: 25100 t: 1152846 rew: 1.7881393e-07
e: 25200 t: 1156713 rew: 0.099000156
e: 25300 t: 1160738 rew: 0.3960001
e: 25400 t: 1163562 rew: -1.9799995
e: 25500 t: 1168749 rew: -1.8809996
e: 25600 t: 1172178 rew: 0.3960001
e: 25700 t: 1176448 rew: 0.3960001
e: 25800 t: 1180231 rew: -0.49499977
e: 25900 t: 1183265 rew: 0.2970001
e: 26000 t: 1185743 rew: -3.6629994
e: 26100 t: 1188817 rew: 0.19800013
e: 26200 t: 1190677 rew: 0.19800013
e: 26300 t: 1192658 rew: 0.3960001

In [ ]:
bandwidth = 100
sampling = 1000
# tot_time = 100000

LA0 = np.zeros((2,int(tot_time/sampling )-bandwidth))
lookahead = 0
for r in range(2):
    round_ = r+1
    env_name = 'Grid'
    env_name = '%s_LA_%d_W%d_H%d_round%d' %(env_name,lookahead,w,h,round_)

    fnam = './data/reward%s.npy' %(env_name)
    tot_3 = np.load(fnam)
    total_3_sample= np.zeros(int(tot_time/sampling ))
    total_3= np.zeros(int(tot_time/sampling )-bandwidth)
    for i in range(int(tot_time/sampling)):
        total_3_sample[i] = np.sum(tot_3[sampling*i:sampling*i+sampling])/sampling
    for i in range(len(total_3_sample)-bandwidth):
        total_3[i] = np.sum(total_3_sample[i:i+bandwidth])/bandwidth

    LA0[r,:] = total_3


In [ ]:
LA1 = np.zeros((1,int(tot_time/sampling )-bandwidth))
lookahead = 1
for r in range(1):
    round_ = r+1
    env_name = 'Grid'
    env_name = '%s_LA_%d_W%d_H%d_round%d' %(env_name,lookahead,w,h,round_)
    fnam = './data/reward%s.npy' %(env_name)
    tot_3 = np.load(fnam)
    total_3_sample= np.zeros(int(tot_time/sampling ))
    total_3= np.zeros(int(tot_time/sampling )-bandwidth)
    for i in range(int(tot_time/sampling)):
        total_3_sample[i] = np.sum(tot_3[sampling*i:sampling*i+sampling])/sampling
    for i in range(len(total_3_sample)-bandwidth):
        total_3[i] = np.sum(total_3_sample[i:i+bandwidth])/bandwidth

    LA1[r,:] = total_3



In [ ]:
fonts = 15
fram = np.linspace(1,tot_time,int(tot_time/sampling ),dtype=np.int32)[0:-bandwidth]
plt.ticklabel_format(style='sci', axis='x')
plt.ticklabel_format(axis='both', style='sci', scilimits=(-2,2))
plt.legend(fontsize=fonts)

# ax = sns.tsplot(data=LA0,time=fram,value='Value',color = 'b',condition = "LA = 0")
ax = sns.tsplot(data=LA1,time=fram,value='Value',color = 'c',condition = "LA = 1")

plt.xlabel("Number of steps",fontsize=fonts, family = 'serif')
plt.ylabel("Average reward per episode",fontsize=fonts, family = 'serif')
plt.title("%s" %(env_name),fontsize=fonts, family = 'serif')
# ax.set(xlabel='Number of steps', ylabel='Average reward per episode')

# plt.savefig("Pong_reward_lambda_3_runs.png",format='png', dpi=2000)
# plt.savefig("Pong_reward_lambda_3_runs.eps",format='eps', dpi=2000)

In [ ]:
s = torch.zeros((1,2),dtype=torch.int32)
s[0,0] = 3
s[0,1] = 3
ss = torch.zeros((1,2),dtype=torch.int32)
ss[0,0] = 0
ss[0,1] = 0
print(Q(s.type(dtype))-Q(ss.type(dtype)))

In [ ]:
s = torch.zeros((1,2),dtype=torch.int32)
s[0,0] = 3
s[0,1] = 3
Q(s.type(dtype))

In [ ]:
s = torch.zeros((1,2),dtype=torch.int32)
s[0,0] = 0
s[0,1] = 0
Q(s.type(dtype))

In [ ]:
current_nextstates= states[choices+1].unsqueeze(1).type(dtype)
Target_Q(current_nextstates).max(1)[0].detach()

In [ ]:
d_error

In [ ]:
np.linspace(0,batchsize-1,batchsize,dtype=np.int32)

In [ ]:
rewards

In [ ]:
np.linspace(0,batchsize-1,batchsize,dtype=np.int32)

In [ ]:
current_value

In [ ]:
current_rewards

In [ ]:
(current_rewards==0)*gamma*Target_Q(current_nextstates).max(1)[0].detach()

In [ ]:
rewards

In [ ]:
# current_rewards

In [ ]:
current_rewards[0]==life_cost

In [ ]:
current_rewards == -1 * life_cost

In [ ]:
current_rewards[0]

In [ ]:
choices = np.random.choice(time, batchsize)
current_action = actions[choices].unsqueeze(1)
current_rewards = rewards[choices].type(dtype)
indecies = np.linspace(0,batchsize-1,batchsize,dtype=np.int32)
current_nextstates = states[choices+1].unsqueeze(1).type(dtype)
current_value = Q(states[choices].unsqueeze(1).type(dtype))[indecies,current_action.squeeze(1).type(torch.int32).numpy()]
loss = current_value-current_rewards-(current_rewards== -1 * life_cost).type(dtype)*gamma*Target_Q(current_nextstates).max(1)[0].detach()
d_error = loss.clamp(-1, 1)
optimizer.zero_grad()
current_value.backward(d_error.data)
# optimizer.step()

In [ ]:
d_error.shape